In [1]:
from pathlib import Path

from tqdm import trange

from agent import Agent
from prompts import ANTIBODIES_CONTEXTS, ESM_ANTIBODIES_PAPER, PRINCIPAL_INVESTIGATOR
from run_individual_meeting import run_individual_meeting
from run_scientific_meeting import run_scientific_meeting
from utils import load_summaries

In [2]:
# Set up key parameters
team_lead = PRINCIPAL_INVESTIGATOR
num_iterations = 3
num_rounds = 3
save_dir = Path("antibody_design")
model = "gpt-4o"
discussion_paths = []
contexts = ANTIBODIES_CONTEXTS

In [3]:
# Ask PI to select team members
ANTIBODIES_TEAM_PROMPT = "You are working on a project to develop antibodies for the SARS-CoV-2 spike protein, ideally for the newest variant of the virus and with broad spectrum activity across variants. You need to select a team of scientists to help you with this project. Please select the team members you would like to invite to a discussion to design the antibody discovery approach. For each team member, please specify the following: 1. Title, 2. Expertise, 3. Goal, and 4. Role. Please use your own description as an example."

In [4]:
# Select team members
for iteration_num in trange(num_iterations, desc="Project Iterations"):
    run_individual_meeting(
        team_member=team_lead,
        agenda=ANTIBODIES_TEAM_PROMPT,
        save_dir=save_dir / "team_selection",
        save_name=f"discussion_{iteration_num + 1}",
        contexts=contexts,
        model=model,
    )

Project Iterations:  33%|███▎      | 1/3 [00:16<00:32, 16.43s/it]

Input token count: 194
Output token count: 704
Max token length: 898
Cost: $0.01
Time: 0:16


Project Iterations:  67%|██████▋   | 2/3 [00:29<00:14, 14.23s/it]

Input token count: 194
Output token count: 623
Max token length: 817
Cost: $0.01
Time: 0:12


Project Iterations: 100%|██████████| 3/3 [00:41<00:00, 13.73s/it]

Input token count: 194
Output token count: 622
Max token length: 816
Cost: $0.01
Time: 0:12


In [4]:
# Add team members based on discussion 2
COMPUTATIONAL_BIOLOGIST = Agent(
    title="Computational Biologist",
    expertise="bioinformatics, protein structure prediction, and molecular dynamics simulations",
    goal="to identify potential antibody candidates using computational methods and predict their binding affinity to the SARS-CoV-2 spike protein",
    role="to run in silico screenings of antibody libraries, modeling antibody-spike protein interactions, and providing a shortlist of promising candidates for experimental validation",
)
IMMUNOLOGIST = Agent(
    title="Immunologist",
    expertise="immune response mechanisms, antibody generation, and characterization",
    goal="to guide the selection of antibody candidates based on immunological principles and ensure the candidates have the potential for broad-spectrum activity",
    role="to provide insights into the immune response to SARS-CoV-2, help design the antibody generation strategy, and interpret the results from binding and neutralization assays",
)
STRUCTURAL_BIOLOGIST = Agent(
    title="Structural Biologist",
    expertise="X-ray crystallography, cryo-electron microscopy (cryo-EM), and protein structure determination",
    goal="to determine the high-resolution structures of antibody-spike protein complexes to understand the binding mechanisms and improve antibody design",
    role="to solve the structures of selected antibody-spike protein complexes and providing structural insights to refine antibody candidates",
)
VIROLOGIST = Agent(
    title="Virologist",
    expertise="SARS-CoV-2 biology, viral entry mechanisms, and neutralization assays",
    goal="to validate the neutralizing activity of antibody candidates against SARS-CoV-2 and its variants",
    role="to oversee the design and execution of neutralization assays, interpret the results, and ensure that the selected antibodies are effective against multiple variants of the virus",
)
DATA_SCIENTIST = Agent(
    title="Data Scientist",
    expertise="machine learning, data analysis, and predictive modeling",
    goal="to develop and apply machine learning models to predict the efficacy and broad-spectrum activity of antibody candidates",
    role="to analyze experimental data, develop predictive models, and integrate data from various sources to guide the selection and optimization of antibody candidates",
)
EXPERIMENTAL_BIOLOGIST = Agent(
    title="Experimental Biologist",
    expertise="antibody engineering, protein expression, and purification",
    goal="to produce and characterize the antibody candidates identified through computational and immunological methods",
    role="to express, purify, and initially characterize antibody candidates, as well as to prepare samples for binding and neutralization assays",
)

team_members = (
    COMPUTATIONAL_BIOLOGIST,
    IMMUNOLOGIST,
    STRUCTURAL_BIOLOGIST,
    VIROLOGIST,
    DATA_SCIENTIST,
    EXPERIMENTAL_BIOLOGIST,
)

In [5]:
ANTIBODIES_PROMPT = "You are working on a project to develop antibodies for the SARS-CoV-2 spike protein, ideally for the newest variant of the virus and with broad spectrum activity across variants. Please design a specific antibody discovery approach for this target that uses machine learning to design antibody candidates. Decide what specific machine learning model to use and precisely how it will be used. If the model needs to be trained, please decide on a specific dataset for training. If the model is pre-trained, please explain exactly how it will be used for identifying or designing antibody candidates."

In [6]:
ANTIBODIES_QUESTIONS = (
    "Will you design the antibodies de novo or will you modify existing antibodies?",
    "Will you train a model from scratch or use a pre-trained model?",
    "What specific model architecture will you use?",
    "If training a model, what dataset will you use for training?",
    "How exactly will you use your model to design antibodies?"
)

In [ ]:
# Run antibodies project design
for iteration_num in trange(num_iterations, desc="Project Iterations"):
    run_scientific_meeting(
        team_lead=team_lead,
        team_members=team_members,
        agenda=ANTIBODIES_PROMPT,
        agenda_questions=ANTIBODIES_QUESTIONS,
        contexts=contexts,
        save_dir=save_dir / "project_design",
        save_name=f"discussion_{iteration_num + 1}",
        num_rounds=num_rounds,
        model=model,
    )

In [7]:
ESM_PROMPT = "You are working on a project to develop antibodies for the SARS-CoV-2 spike protein, ideally for the newest variant of the virus and with broad spectrum activity across variants. You will use the ESM family of models for antibody design. Please design a method for applying ESM to this antibody design problem. Specify the exact model you will use and how you will use it to design antibodies."

In [8]:
ESM_QUESTIONS = (
    "Which ESM model will you use?"
    "Will you design the antibodies de novo or will you modify existing antibodies?",
    "If modifying existing antibodies, which precise antibody or antibodies will you modify?",
    "How exactly will you use your model to design antibodies?",
    "What is the precise process for designing antibodies and selecting candidates for experimental validation?",
    "How will you computationally evaluate the quality of the design antibodies?",
    "What objectives will you optimize for in the design process?",
)

In [ ]:
# Run ESM project design
for iteration_num in trange(num_iterations, desc="Project Iterations"):
    run_scientific_meeting(
        team_lead=team_lead,
        team_members=team_members,
        agenda=ESM_PROMPT,
        agenda_questions=ESM_QUESTIONS,
        contexts=contexts,
        save_dir=save_dir / "esm",
        save_name=f"discussion_{iteration_num + 1}",
        num_rounds=num_rounds,
        model=model,
    )

In [9]:
# Select preferred summary
discussion_paths.append(save_dir / "esm" / "discussion_2.json")

In [10]:
# Load summaries
summaries = load_summaries(discussion_paths=discussion_paths)

In [11]:
ESM_IMPLEMENT_PROMPT = "You now need to implement an ESM-based approach for antibody design. Please write code to implement the method you designed in the previous discussion. Your code must be self-contained (with appropriate imports), and you must explain how the code works."

In [ ]:
# Run ESM implement
for iteration_num in trange(num_iterations, desc="Project Iterations"):
    run_individual_meeting(
        team_member=DATA_SCIENTIST,
        agenda=ESM_IMPLEMENT_PROMPT,
        summaries=summaries,
        contexts=contexts,
        save_dir=save_dir / "esm_implement",
        save_name=f"discussion_{iteration_num + 1}",
        model=model,
    )

In [12]:
ESM_REIMPLEMENT_PROMPT = "Above is the full text from a recent scientifc paper describing the use of ESM, a pre-trained protein language model, to design antibodies. Please read the paper and then describe in detail how to reproduce the results in the paper and what steps you would take next to build upon this work."

In [13]:
ESM_REIMPLEMENT_QUESTIONS = (
    "How would you reproduce the results in the paper? Please provide a detailed, step-by-step set of instructions. Please copy as much text/code as needed from the discussion in order to provide a complete, self-contained answer here.",
    "How would you reimplement the computational parts of the paper? Please provide self-contained Python code that implements the main experiments from the paper. Please copy as much text/code as needed from the discussion in order to provide a complete, self-contained answer here.",
    "What lab experiments would you run to validate the computationally designed antibodies? Please provide a step-by-step lab protocol. Please copy as much text/code as needed from the discussion in order to provide a complete, self-contained answer here.",
    "How would you extend beyond this paper to further advance the science of protein language models for antibody design? Please copy as much text/code as needed from the discussion in order to provide a complete, self-contained answer here.",
)

In [ ]:
# Run ESM reimplement
for iteration_num in trange(num_iterations, desc="Project Iterations"):
    run_scientific_meeting(
        team_lead=team_lead,
        team_members=team_members,
        agenda=ESM_REIMPLEMENT_PROMPT,
        agenda_questions=ESM_REIMPLEMENT_QUESTIONS,
        contexts=contexts + (ESM_ANTIBODIES_PAPER,),
        save_dir=save_dir / "esm_reimplement",
        save_name=f"discussion_{iteration_num + 1}",
        num_rounds=num_rounds,
        model=model,
    )

Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team:   0%|          | 0/7 [00:00<?, ?it/s]

Team:  14%|█▍        | 1/7 [00:15<01:33, 15.56s/it]

Team:  29%|██▊       | 2/7 [00:46<02:02, 24.48s/it]

Team:  43%|████▎     | 3/7 [01:17<01:50, 27.64s/it]

Team:  57%|█████▋    | 4/7 [01:37<01:13, 24.57s/it]

Team:  71%|███████▏  | 5/7 [01:56<00:44, 22.43s/it]

Team:  86%|████████▌ | 6/7 [02:23<00:24, 24.02s/it]

Team: 100%|██████████| 7/7 [02:43<00:00, 23.29s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [02:43<08:09, 163.04s/it]

Team:   0%|          | 0/7 [00:00<?, ?it/s]

Team:  14%|█▍        | 1/7 [00:23<02:21, 23.50s/it]

Team:  29%|██▊       | 2/7 [01:03<02:45, 33.17s/it]

Team:  43%|████▎     | 3/7 [01:21<01:45, 26.45s/it]

Team:  57%|█████▋    | 4/7 [01:39<01:08, 22.91s/it]

Team:  71%|███████▏  | 5/7 [02:04<00:47, 23.68s/it]

Team:  86%|████████▌ | 6/7 [02:34<00:25, 25.92s/it]

Team: 100%|██████████| 7/7 [03:01<00:00, 25.92s/it]

Rounds (+ Summary Round):  50

Input token count: 824,532
Output token count: 19,205
Max token length: 48,449
Cost: $4.41
Time: 9:44



Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Team:   0%|          | 0/7 [00:00<?, ?it/s]

Team:  14%|█▍        | 1/7 [00:40<04:04, 40.71s/it]

Team:  29%|██▊       | 2/7 [01:04<02:33, 30.67s/it]

Team:  43%|████▎     | 3/7 [1:29:10<2:42:01, 2430.29s/it]

Team:  57%|█████▋    | 4/7 [1:29:31<1:13:57, 1479.32s/it]

Team:  71%|███████▏  | 5/7 [1:29:56<31:49, 954.65s/it]   

Team:  86%|████████▌ | 6/7 [1:30:23<10:39, 639.57s/it]

Team: 100%|██████████| 7/7 [1:30:53<00:00, 779.03s/it]

Rounds (+ Summary Round):  25%|██▌       | 1/4 [1:30:53<4:32:39, 5453.22s/it]

Team:   0%|          | 0/7 [00:00<?, ?it/s]

Team:  14%|█▍        | 1/7 [00:24<02:28, 24.79s/it]

Team:  29%|██▊       | 2/7 [31:38<1:32:43, 1112.67s/it]

Team:  43%|████▎     | 3/7 [31:54<40:46, 611.69s/it]   

Team:  57%|█████▋    | 4/7 [32:13<18:53, 377.75s/it]

Team:  71%|███████▏  | 5/7 [32:35<08:19, 249.51s/it]

Team:  86%|████████▌ | 6/7 [33:06<02:55, 175.22s/it]

Team: 100%|██████████| 7/7 [33:37<00:00, 